<a href="https://colab.research.google.com/github/abhaswanchu1/mids-281-final-project-cars/blob/main/281_Data_Loading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
from shutil import copyfile
from sklearn.model_selection import train_test_split
import cv2
file_location = 'drive/MyDrive/' #you'll have to change this to your local directory

In [ ]:
# File locations
excel_file = 'drive/MyDrive/cars_classes.xlsx'
train_folder = 'drive/MyDrive/cars_train'
test_folder = 'drive/MyDrive/cars_test'
output_folder = 'drive/MyDrive/cars_dataset'  # Folder for the combined dataset

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

In [ ]:
# Load training data
train_df = pd.read_excel(excel_file, sheet_name='train')

# Load testing data
test_df = pd.read_excel(excel_file, sheet_name='test')

In [ ]:
# Function to label and copy images
def label_and_copy(df, image_folder, output_folder):
    for index, row in df.iterrows():
        image_name = row['image']  # Assuming 'image_name' column exists
        true_class_name = row['true_class_name']
        class_name = true_class_name.split()[0]
        source_path = os.path.join(image_folder, image_name)
        destination_path = os.path.join(output_folder, class_name, image_name)
        os.makedirs(os.path.join(output_folder, class_name), exist_ok=True)  # Create class folders
        copyfile(source_path, destination_path)

# Label and copy training images
label_and_copy(train_df, train_folder, output_folder)

# Label and copy testing images
label_and_copy(test_df, test_folder, output_folder)

#this will take a while

In [ ]:
def create_filtered_dataframe(dataset_path, brands):
  """
  Creates a Pandas DataFrame from a dataset folder,
  filtering for specific brands.

  Args:
    dataset_path: Path to the dataset folder.
    brands: List of brands to include.

  Returns:
    A Pandas DataFrame with image paths and labels.
  """

  data = []
  for brand in brands:
    brand_folder = os.path.join(dataset_path, brand)
    if os.path.isdir(brand_folder):
      for image_name in os.listdir(brand_folder):
        image_path = os.path.join(brand_folder, image_name)
        image = cv2.imread(image_path)
        data.append({'image_path': image_path, 'brand': brand, 'image':image})

  return pd.DataFrame(data)

# Specify the dataset path and desired brands
cars_dataset_path = 'drive/MyDrive/cars_dataset'
selected_brands = ['Chevrolet', 'BMW', 'Dodge', 'Audi']

# Create the filtered DataFrame
filtered_df = create_filtered_dataframe(cars_dataset_path, selected_brands)

# Display the DataFrame (optional)
filtered_df.head()

In [ ]:
# Split the DataFrame into training and testing sets
train_df, test_df = train_test_split(filtered_df, test_size=0.3, random_state=42,stratify=filtered_df['brand'])

In [ ]:
def convert_to_grayscale(image):
       """Converts an image to grayscale using OpenCV."""
       return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [ ]:
train_df['image'] = train_df['image'].apply(convert_to_grayscale)
test_df['image'] = test_df['image'].apply(convert_to_grayscale)

In [ ]:
# Print the shapes of the resulting DataFrames
print("Train DataFrame shape:", train_df.shape)
print("Test DataFrame shape:", test_df.shape)